# GEOGRAPHIC PARTS
# IMPORT LIBS

In [16]:
import pandas as pd
import plotly.express as px
import math
import numpy as np
import geopy.distance

# OPEN DATASET

In [17]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_00_dropna.csv')
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


# DELETE ALL LATITUDE OUT OF RANGE

In [18]:
df = df.loc[(df['pickup_latitude'] >= 40 ) & (df['pickup_latitude'] <= 42 ) & (df['dropoff_latitude'] >= 40 ) & (df['dropoff_latitude'] <= 42 ) & (df['pickup_longitude'] >= -76 ) & (df['pickup_longitude'] <= -72 ) & (df['dropoff_longitude'] >= -76 ) & (df['dropoff_longitude'] <= -72 )]
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


# CALCUL DISTANCE WITH GEOPY LIB

In [19]:
df['Distance (miles)'] = df.apply(lambda row: geopy.distance.geodesic((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])).miles, axis=1)

df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance (miles)
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.639764
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,5.246511
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,0.863167
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,1.737223
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.241710


On this plot, we can see too many drive with high distance but cheap price AND too many drive with cheap distance but high price.
So, we gonna limit this by adding a price by kmfeature and try to drpo rows with price by km useless

# VISUALISATION OF OUTLIERS
## DISPLAY DISTANCE DISTRIBUTION

In [20]:
# fig = px.violin(df, y="Distance (miles)", box=True,
#                 # points="all",
#                 # hover_data=df.columns,
#                 title='Violin plot of Distance distribution'
#                 )
# fig.show()

# Calcul D’Outliers :
X < q1 - 1.5 x iqr X > q3 - 1.5 x iqr

In [21]:
q1 = df['Distance (miles)'].quantile([.25])
q1 = q1[0.25]
q3 = df['Distance (miles)'].quantile([.75])
q3 = q3[0.75]

iqr = q3 - q1

In [22]:
print(f" IQR = {iqr}, q1 = {q1}, q3 = {q3}")

 IQR = 1.6539670736319074, q1 = 0.7792571066611934, q3 = 2.433224180293101


In [23]:
outliers_superieur = q3 + (1.5 * iqr)
outliers_inferieur = q1 - (1.5 * iqr)

print(f'Outliers Inférieur = {outliers_inferieur}')
print(f'Outliers Supérieur = {outliers_superieur}')

Outliers Inférieur = -1.7016935037866676
Outliers Supérieur = 4.914174790740962


# DROP OUTLIERS

In [24]:
df = df.loc[(df['Distance (miles)'] <= outliers_superieur) & (df['Distance (miles)'] >= outliers_inferieur)]
df.shape

(1789526, 9)

# DISPLAY NEW VIOLIN PLOT WITHOUT OUTLIERS

In [25]:
# fig = px.violin(df, y="Distance (miles)", box=True,
#                 # points="all",
#                 # hover_data=df.columns,
#                 title='Violin plot of Distance distribution'
#                 )
# fig.show()

# SCATTER PLOT OF FARE AMOUNT BY DISTANCE

In [26]:
# fig = px.scatter(df, x='Distance (miles)', y='fare_amount', color='passenger_count')
# fig.show()

# EXPORT CSV

In [27]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv', index=False)